# Student name, ID: Morteza Layegh Mirhosseini (220003166) - PG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
    import torch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

cuda


# TO DO (IMPORTANT

In [ ]:
import os
# TODO: Fill in the Google Drive path where you uploaded the shared folder
# Example: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'coursework'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NLP/coursework' # Change this line
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

#you should see the following in the out put: 
#['coursework .ipynb', 'trained_models', 'dataset', 'feature_models', 'Test .ipynb', 'README.txt']

['trained_models', 'dataset', 'feature_models', 'README.txt', 'Test .ipynb', 'coursework .ipynb']


# Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score ,accuracy_score
from sklearn.svm import SVC
import spacy
import gensim
import re
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

#from datasets import load_dataset
from gensim.models import Word2Vec, KeyedVectors

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pickle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.2 MB/s eta 0:00:00


# Load data

In [ ]:
# load data 
dataset_path = os.path.join( GOOGLE_DRIVE_PATH,'dataset','Data_20000.tsv')
dataset = pd.read_csv(dataset_path, sep='\t')

#Extract input and output data
X = dataset['review']
y = dataset['label']


In [ ]:
# Check the class balance
class_counts = dataset['label'].value_counts()
class_balance = class_counts / len(dataset) * 100

# Print the class balance
print("Class Balance:")
for class_label, class_count in class_counts.items():
    print("Class {}: {} samples ({:.2f}%)".format(class_label, class_count, class_balance[class_label]))

Class Balance:
Class 1: 10257 samples (51.29%)
Class 0: 9743 samples (48.72%)


# Pre-Processing Function


In [ ]:


def preprocess_text(text):
    
    text = text.lower()
    
    text = re.sub(r'http\S+', '', text)
    
    text = re.sub(r'\S+@\S+', '', text)
    
    text = re.sub(r'([aeiou])\1{2,}', r'\1', text)
    
    # Replace negative constructs with "not"
    text = re.sub(r"can\'t|cannot", "can not", text)
    text = re.sub(r"don\'t|doesn\'t", "do not", text)
    text = re.sub(r"won\'t|wouldn\'t", "will not", text)
    text = re.sub(r"shouldn\'t|couldn\'t", "should not", text)
    text = re.sub(r"haven\'t|hasn\'t", "have not", text)
    text = re.sub(r"hadn\'t|wasn\'t|weren\'t|isn\'t|aren\'t", "not", text)
    text = re.sub(r"never|none|nothing|nowhere|n\'t", "not", text)
    
    # Replace emoticons with "positive" or "negative"
    text = re.sub(r"(:\)|:-\)|:\]|:-\])+", "positive", text)
    text = re.sub(r"(:\(|:<|\[|\]-\(|:-\[|:-\(|:-{|:-/|:-\\)+", "negative", text)
    
    # punctuations and whitespace characters
    #stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    words = [word for word in words if word.isalnum()]


    text = ' '.join(words)

    return text


In [ ]:
text = "This is an example of text that needs to be preprocessed. It cannot includes a URL (https://example.com), an email address (john@example.com), and some emoticons :), :-), :(, and :-("
preprocessed_text = preprocess_text(text)
print(preprocessed_text)

this is an example of text that needs to be preprocessed it can not includes a url an email address and some emoticons positive positive negative and negative


# Train Functions

In [ ]:
def train_svm(X_train, y_train):

  # Create an instance of the SVM classifier
  clf = SVC(C=1 , kernel='linear')

  # Fit the clf instance to the training data
  clf.fit(X_train, y_train)

  return clf

def train_mlp(X_train, y_train):

  # Create an instance of the MLP classifier
  clf = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', max_iter=5, verbose=True)

  # Fit the clf instance to the training data
  clf.fit(X_train, y_train)

  return clf

# Evaluation Function:


In [ ]:
def evaluate_model(model, X_test, y_test):

    y_pred = model.predict(X_test)

    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Accuracy Score: {:.2f}".format((accuracy_score(y_test, y_pred))))    
    print("Precision Score: {:.2f}".format((precision_score(y_test, y_pred))))
    print("Recall Score: {:.2f}".format((recall_score(y_test, y_pred))))
    print("F1 Score: {:.2f}".format((f1_score(y_test, y_pred))))

# BOW



## BOW + SVM

### preprocess

In [ ]:
# function is adapted from the City NLP Lab (Text Classification Advanced)
def custom_feature_extraction(text, ngrams=3, compute=0, vectorizer=None):
    if compute:
    # Extract unigrams, bigrams, and trigrams
        vectorizer = CountVectorizer(ngram_range=(1, ngrams),preprocessor= preprocess_text)
        features = vectorizer.fit_transform(text)
        return features, vectorizer
    else: 
        features = vectorizer.transform(text)
        return features
    

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=123)



# Extract features using custom feature extraction function
X_train, feat_extractor = custom_feature_extraction(X_train, compute=1)
X_val = custom_feature_extraction(X_val, compute=0, vectorizer=feat_extractor)
X_test = custom_feature_extraction(X_test, compute=0, vectorizer=feat_extractor)

### train

In [ ]:
clf = train_svm(X_train, y_train)

### validation score 

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 86.71%


### save

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','BOW3_SVM.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)


### load & test

In [ ]:
#evaluate 

# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','BOW3_SVM.pkl')

# Load the model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)

#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      1951
           1       0.86      0.87      0.87      2049

    accuracy                           0.86      4000
   macro avg       0.86      0.86      0.86      4000
weighted avg       0.86      0.86      0.86      4000

Confusion Matrix:
 [[1670  281]
 [ 261 1788]]
Accuracy Score: 0.86
Precision Score: 0.86
Recall Score: 0.87
F1 Score: 0.87


## BOW +  MLP

### train

In [ ]:
clf = train_mlp(X_train, y_train)

Iteration 1, loss = 0.40833812
Iteration 2, loss = 0.01333898
Iteration 3, loss = 0.00208847
Iteration 4, loss = 0.00136369
Iteration 5, loss = 0.00114387


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


### validation score

In [ ]:
y_pred= clf.predict(X_val)
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 88.25%


### save 

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','BOW3_MLP.pkl')

# Save the GridSearchCV object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','BOW3_MLP.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)

#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      1951
           1       0.89      0.86      0.87      2049

    accuracy                           0.87      4000
   macro avg       0.87      0.87      0.87      4000
weighted avg       0.87      0.87      0.87      4000

Confusion Matrix:
 [[1727  224]
 [ 293 1756]]
Accuracy Score: 0.87
Precision Score: 0.89
Recall Score: 0.86
F1 Score: 0.87


# TF-IDF

## TF-IDF + SVM

### preprocessing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# function is adapted from the City NLP Lab (Text Classification Advanced)

def custom_feature_extraction(text, ngrams=2, compute=0, vectorizer=None):
    if compute:
        # Extract unigrams, bigrams, and trigrams
        vectorizer = TfidfVectorizer(ngram_range=(1, ngrams),preprocessor= preprocess_text)
        features = vectorizer.fit_transform(text)
        return features, vectorizer
    else: 
        features = vectorizer.transform(text)
        return features

  
# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=123)



# Extract features using custom feature extraction function
X_train, feat_extractor = custom_feature_extraction(X_train, compute=1)
X_val = custom_feature_extraction(X_val, compute=0, vectorizer=feat_extractor)
X_test = custom_feature_extraction(X_test, compute=0, vectorizer=feat_extractor)

### train 

In [ ]:
clf = train_svm(X_train, y_train)


### validation score 

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 88.29%


### save

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TFIDF_SVM.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:
#evaluate 

# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TFIDF_SVM.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)


Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.88      0.87      1951
           1       0.88      0.87      0.88      2049

    accuracy                           0.88      4000
   macro avg       0.87      0.88      0.87      4000
weighted avg       0.88      0.88      0.88      4000

Confusion Matrix:
 [[1718  233]
 [ 267 1782]]
Accuracy Score: 0.88
Precision Score: 0.88
Recall Score: 0.87
F1 Score: 0.88


## TF-IDF + MLP

### train

In [ ]:
clf = train_mlp(X_train, y_train)

Iteration 1, loss = 0.57570878
Iteration 2, loss = 0.09410281
Iteration 3, loss = 0.00890426
Iteration 4, loss = 0.00376778
Iteration 5, loss = 0.00236847


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


 ### validation score 

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 88.33%


### save 

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TFIDF_MLP.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:

# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TFIDF_MLP.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.87      0.87      1951
           1       0.88      0.88      0.88      2049

    accuracy                           0.88      4000
   macro avg       0.88      0.88      0.88      4000
weighted avg       0.88      0.88      0.88      4000

Confusion Matrix:
 [[1705  246]
 [ 242 1807]]
Accuracy Score: 0.88
Precision Score: 0.88
Recall Score: 0.88
F1 Score: 0.88


# Glove



## Glove  model:(300 dimentions) + SVM 

###preprocess

In [ ]:
import urllib.request
import zipfile
from pathlib import Path

feature_model_path = os.path.join( GOOGLE_DRIVE_PATH,'feature_models','glove.6B.300d.txt')

# Load the embeddings into a dictionary
embeddings = {}
with open(feature_model_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embeddings[word] = vector

In [ ]:

# Apply the preprocess_text function to the 'review' column
dataset['cleaned_review'] = dataset['review'].apply(preprocess_text)


In [ ]:
# Preprocess the data
X = []
for review in dataset['cleaned_review']:
    words = review.split()
    features = []
    for word in words:
        if word in embeddings:
            features.append(embeddings[word])
    if len(features) > 0:
        features = np.mean(features, axis=0)
        X.append(features)
X = np.array(X)
y = dataset['label']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=123)

### train

In [ ]:
clf = train_svm(X_train, y_train)


### validation score

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))

Accuracy Score: 78.38%


### save 

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','Glove300_SVM.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & evaluate

In [ ]:
#evaluate 

# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','Glove300_SVM.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.82      0.80      1951
           1       0.82      0.78      0.80      2049

    accuracy                           0.80      4000
   macro avg       0.80      0.80      0.80      4000
weighted avg       0.80      0.80      0.80      4000

Confusion Matrix:
 [[1599  352]
 [ 446 1603]]
Accuracy Score: 0.80
Precision Score: 0.82
Recall Score: 0.78
F1 Score: 0.80


## Glove  model:(300 dimentions) + MLP 

### train

In [ ]:
clf = train_mlp(X_train, y_train)

Iteration 1, loss = 0.62764823
Iteration 2, loss = 0.49648416
Iteration 3, loss = 0.46079274
Iteration 4, loss = 0.44831388
Iteration 5, loss = 0.44275829


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


### validation score


In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 78.25%


### save 

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','Glove300_MLP.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:
#evaluate 

# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','Glove300_MLP.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.76      0.78      1951
           1       0.78      0.82      0.80      2049

    accuracy                           0.79      4000
   macro avg       0.79      0.79      0.79      4000
weighted avg       0.79      0.79      0.79      4000

Confusion Matrix:
 [[1489  462]
 [ 366 1683]]
Accuracy Score: 0.79
Precision Score: 0.78
Recall Score: 0.82
F1 Score: 0.80


# Word2vec Gensim 

### adapted from these toturals
https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381

https://www.youtube.com/watch?v=Q2NtCcqmIww

https://www.youtube.com/watch?v=Q2NtCcqmIww

## Trained word2vec + SVM 


### training the word2vec model




In [ ]:

# Apply the preprocess_text function to the 'review' column
dataset['cleaned_review'] = dataset['review'].apply(preprocess_text)

In [ ]:
# Function to convert a string review into a list of words
def split_review(review):
    return review.split()

# Apply the function to the "review" column in the DataFrame
dataset["cleaned_review_words"] = dataset["cleaned_review"].apply(split_review)

In [ ]:
#Extract input and output data
X = dataset['cleaned_review_words']
y = dataset['label']


In [ ]:
def train_word2vec(review_text):

  #Training the Word2Vec Model
  model = gensim.models.Word2Vec(
      vector_size=150,
      window=10,
      min_count=2,
      workers=4,
  )

  # build vocab
  model.build_vocab(review_text, progress_per=1000)

  # train model
  model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

  return model 



In [ ]:
# splitting before training the word2vec
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# training the model
model = train_word2vec(X_train)


### save word2vec model

In [ ]:
#save model
feature_model_path = os.path.join( GOOGLE_DRIVE_PATH,'feature_models','word2vec_gensim_amazonReview.model')
model.save(feature_model_path)

In [ ]:
#load model
feature_model_path = os.path.join( GOOGLE_DRIVE_PATH,'feature_models','word2vec_gensim_amazonReview.model')
model = gensim.models.Word2Vec.load(feature_model_path)


### preprocessing

In [ ]:
def To_word2vec(reviews):
# Preprocess the data
  X = []
  for review in reviews:
      #words = review.split()
      features = []
      for word in review:
          if word in model.wv:
              features.append(model.wv[word])
      if len(features) > 0:
          features = np.mean(features, axis=0)
          X.append(features)
  return X


In [ ]:
# creating validation set 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=123)

# extracting the word2vec features for ever review ( mean )
X_train = To_word2vec(X_train)
X_val = To_word2vec(X_val)
X_test = To_word2vec(X_test)

X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

### train classifier

In [ ]:
clf = train_svm(X_train, y_train)


### validation score

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 75.29%


### save

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TRAINED_WORD2VEC__SVM.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:
#evaluate 
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TRAINED_WORD2VEC__SVM.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.77      0.76      1951
           1       0.78      0.75      0.76      2049

    accuracy                           0.76      4000
   macro avg       0.76      0.76      0.76      4000
weighted avg       0.76      0.76      0.76      4000

Confusion Matrix:
 [[1510  441]
 [ 518 1531]]
Accuracy Score: 0.76
Precision Score: 0.78
Recall Score: 0.75
F1 Score: 0.76


## Trained word2vec + MLP 


### train classifier 

In [ ]:
clf = train_mlp(X_train, y_train)

Iteration 1, loss = 0.60168925
Iteration 2, loss = 0.52950450
Iteration 3, loss = 0.51177868
Iteration 4, loss = 0.50435800
Iteration 5, loss = 0.49711545


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


### validation score

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 73.62%


### save

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TRAINED_WORD2VEC__MLP.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:
#evaluate 
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TRAINED_WORD2VEC__MLP.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.85      0.76      1951
           1       0.82      0.64      0.72      2049

    accuracy                           0.74      4000
   macro avg       0.75      0.74      0.74      4000
weighted avg       0.76      0.74      0.74      4000

Confusion Matrix:
 [[1658  293]
 [ 740 1309]]
Accuracy Score: 0.74
Precision Score: 0.82
Recall Score: 0.64
F1 Score: 0.72


# Pre-trained google word2vec

## Google word2vec +SVM


### preprocess

In [ ]:
#load google word2vec model
feature_model_path = os.path.join( GOOGLE_DRIVE_PATH,'feature_models','GoogleNews-vectors-negative300.bin.gz')
model =  KeyedVectors.load_word2vec_format(feature_model_path, binary=True, limit= 100000)

In [ ]:
# Apply the preprocess_text function to the 'review' column
dataset['cleaned_review'] = dataset['review'].apply(preprocess_text)


In [ ]:
# Preprocess the data
X = []
for review in dataset['cleaned_review']:
    words = review.split()
    features = []
    for word in words:
        if word in model:
            features.append(model[word])
    if len(features) > 0:
        features = np.mean(features, axis=0)
        X.append(features)

In [ ]:
X = np.array(X)
y = dataset['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=123)

### train 

In [ ]:
clf = train_svm(X_train, y_train)


### validation score 

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))


Accuracy Score: 81.92%


### save 

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','GOOGLE_WORD2VEC_SVM.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:
#evaluate 
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','GOOGLE_WORD2VEC_SVM.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.84      0.81      1951
           1       0.84      0.79      0.81      2049

    accuracy                           0.81      4000
   macro avg       0.81      0.81      0.81      4000
weighted avg       0.82      0.81      0.81      4000

Confusion Matrix:
 [[1633  318]
 [ 425 1624]]
Accuracy Score: 0.81
Precision Score: 0.84
Recall Score: 0.79
F1 Score: 0.81


## Google word2vec + MLP

### train 

In [ ]:
clf = train_mlp(X_train, y_train)

Iteration 1, loss = 0.63701807
Iteration 2, loss = 0.48284855
Iteration 3, loss = 0.43995167
Iteration 4, loss = 0.42583416
Iteration 5, loss = 0.41739937


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


### validation score 

In [ ]:
y_pred= clf.predict(X_val)
from sklearn.metrics import classification_report, accuracy_score
validation_accuracy = accuracy_score(y_val,y_pred )
print("Accuracy Score: {:.2f}%".format(validation_accuracy * 100))

Accuracy Score: 81.08%


### save

In [ ]:
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','GOOGLE_WORD2VEC_MLP.pkl')

# Save the model object as a pickle file
with open(model_path, 'wb') as file:
    pickle.dump(clf, file)

### load & test

In [ ]:
#evaluate 
# Define the file name for the pickle file
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','GOOGLE_WORD2VEC_MLP.pkl')

# Load the  model from the pickle file
with open(model_path, 'rb') as file:
    clf = pickle.load(file)


#Predict & Evaluate 
evaluate_model(clf, X_test, y_test)

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.85      0.81      1951
           1       0.84      0.77      0.80      2049

    accuracy                           0.81      4000
   macro avg       0.81      0.81      0.81      4000
weighted avg       0.81      0.81      0.81      4000

Confusion Matrix:
 [[1654  297]
 [ 471 1578]]
Accuracy Score: 0.81
Precision Score: 0.84
Recall Score: 0.77
F1 Score: 0.80


# DistilBERT

### Libraries

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Load Data

In [ ]:
dataset_path = os.path.join( GOOGLE_DRIVE_PATH,'dataset','Data_20000.tsv')
dataset = pd.read_csv(dataset_path, sep='\t')


X = dataset['review']
y = dataset['label']

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=123)

#  encode the text data
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_encoded_data = tokenizer(X_train.tolist(), padding=True, truncation=True)
test_encoded_data = tokenizer(X_test.tolist(), padding=True, truncation=True)
val_encoded_data = tokenizer(X_val.tolist(), padding=True, truncation=True)
 
# dataset class
class SentimentDataset(Dataset):
    def __init__(self, encoded_data, labels):
        self.encoded_data = encoded_data
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {key: torch.tensor(text[idx]) for key, text in self.encoded_data.items()}, torch.tensor(self.labels[idx])


train_data = SentimentDataset({key: text for key, text in train_encoded_data.items()}, y_train.tolist())
test_data = SentimentDataset({key: text for key, text in test_encoded_data.items()}, y_test.tolist())
val_data = SentimentDataset({key: text for key, text in val_encoded_data.items()}, y_val.tolist())

# define the data loaders
train_loader = DataLoader(train_data, batch_size=24, shuffle=True)
val_loader = DataLoader(val_data, batch_size=24, shuffle=False)
test_loader = DataLoader(test_data, batch_size=24, shuffle=False)

### Model

In [ ]:

# initialize the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
optimizer = AdamW(model.parameters(), lr=5e-5)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

### Train & Evaluation Loop

In [ ]:
#  adapted from the City NLP Lab (transformes)
# define the training loop
def train_loop(model, train_loader, val_loader, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch[0]["input_ids"].to(device)
            attention_mask = batch[0]["attention_mask"].to(device)
            labels = batch[1].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels) #asl
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        
        # evaluate on the validation set
        model.eval()
        val_losses = []
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch[0]["input_ids"].to(device)
                attention_mask = batch[0]["attention_mask"].to(device)
                labels = batch[1].to(device)
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_losses.append(loss.item())
                val_preds.extend(torch.argmax(outputs.logits, axis=1).tolist())
                val_labels.extend(labels.tolist())
        
        # calculate the metrics and print the results
        acc = accuracy_score(val_labels, val_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(val_labels, val_preds, average="binary")
        print(f"Epoch {epoch+1}: Train Loss = {sum(train_losses)/len(train_losses):.4f} Val Loss = {sum(val_losses)/len(val_losses):.4f}")

###Train

In [ ]:
# define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# move the model to the device
model.to(device)

# train the model
train_loop(model, train_loader, val_loader, optimizer, epochs=3)


Epoch 1: Train Loss = 0.2725 Val Loss = 0.2087
Epoch 2: Train Loss = 0.1323 Val Loss = 0.2429
Epoch 3: Train Loss = 0.0652 Val Loss = 0.2627


### save 

In [ ]:

model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','DISTILBERT.pth')

# Save the trained model to a specific path
torch.save(model.state_dict(), model_path)

### Load

In [ ]:
model_path = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','DISTILBERT.pth')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# move the model to the device
model.to(device)

# Load the model state dictionary from the saved file
model.load_state_dict(torch.load(model_path,map_location=device))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

<All keys matched successfully>

### Test

In [ ]:
# define the test loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# move the model to the device
model.to(device)
def test_loop(model, test_loader):
    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch[0]["input_ids"].to(device)
            attention_mask = batch[0]["attention_mask"].to(device)
            labels = batch[1].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            test_preds.extend(torch.argmax(outputs.logits, axis=1).tolist())
            test_labels.extend(labels.tolist())
    
    # calculate the metrics and print the results
    acc = accuracy_score(test_labels, test_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_preds, average="binary")
    print(f"Test Accuracy = {acc:.4f}")
    print(f"Test Precision = {precision:.4f}")
    print(f"Test Recall = {recall:.4f}")
    print(f"Test F1 Score = {f1:.4f}")
    print("Classification Report:\n", classification_report(test_labels, test_preds))
    print("Confusion Matrix:\n", confusion_matrix(test_labels, test_preds))


# test the model
test_loop(model, test_loader)


cuda
Test Accuracy = 0.9145
Test Precision = 0.9348
Test Recall = 0.8956
Test F1 Score = 0.9148
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91      1951
           1       0.93      0.90      0.91      2049

    accuracy                           0.91      4000
   macro avg       0.91      0.91      0.91      4000
weighted avg       0.92      0.91      0.91      4000

Confusion Matrix:
 [[1823  128]
 [ 214 1835]]


### Error Analysis 

In [ ]:
def test_loop_Error(model, test_loader):
    model.eval()
    test_preds = []
    test_labels = []
    test_losses = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch[0]["input_ids"].to(device)
            attention_mask = batch[0]["attention_mask"].to(device)
            labels = batch[1].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss.item()
            test_losses.extend(loss for _ in range(len(labels)))
            test_preds.extend(torch.argmax(outputs.logits, axis=1).tolist())
            test_labels.extend(labels.tolist())
    
    # get the misclassified samples with the highest loss values
    misclassified = []
    for i, (pred, true, loss) in enumerate(zip(test_preds, test_labels, test_losses)):
        if pred != true:
            misclassified.append((i, X_test.iloc[i], true, pred, loss))
    misclassified_sorted = sorted(misclassified, key=lambda x: x[4], reverse=True)
    
    # print the top 20 misclassified samples with the highest loss values
    for i, sample in enumerate(misclassified_sorted[:50]):
        print(f"Sample {i+1}: True Label = {sample[2]}, Predicted Label = {sample[3]}, Loss = {sample[4]:.4f}\nReview Text: {sample[1]}\n")
test_loop_Error(model, test_loader)

Sample 1: True Label = 0, Predicted Label = 1, Loss = 1.1395
Review Text: A fascinating look at Franklin's life in his own voice. It is unfortunate that he did not narrate the last 30 years of his life as well.

Sample 2: True Label = 0, Predicted Label = 1, Loss = 1.1395
Review Text: This is a french book. French books are, of course, often french, but this one is frencher than most. Ah, Pierrot! Reading this fine french book makes me the sad clown of life. Could Voltaire be any frencher, you ask? I doubt it. I am a sad clown, I say. Even when he criticizes the French he does so in a, how do you say, french way. Oui, oui! So come, bring along your Jerry Lewis video tapes and we will read this very french book together. It is frencher than a Quebecer at a Steisand concert. Frencher than Little Richard at an amfAR banquet. Frencher than Richard Simmons at a french pride parade. Read Candide and you too can be a very french sad clown. Sacre Bleu!!!

Sample 3: True Label = 1, Predicted La